In [2]:
import sys
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.metrics import log_loss, brier_score_loss, accuracy_score
from sklearn.calibration import calibration_curve
import torch
import warnings

# Add current directory to path
sys.path.insert(0, '.')

# Import custom modules
from features import TennisFeatureExtractor
from ml_models.neural_network import (
    SymmetricNeuralNetwork,
    NeuralNetworkTrainer,
    train_nn_ensemble,
    predict_ensemble,
    calculate_permutation_importance
)

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

print("✅ Libraries loaded")
print(f"PyTorch version: {torch.__version__}")
print(f"Training started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ Libraries loaded
PyTorch version: 2.9.1
Training started at: 2025-12-28 04:39:30


## 1. Load and Prepare Data

In [3]:
# Load pre-extracted features from logistic regression training
# This avoids recomputing features which takes ~2 minutes

# Connect to database
conn = sqlite3.connect('tennis_data.db')

# Load matches
query = """
SELECT 
    m.match_id,
    m.tournament_date,
    m.surface,
    m.winner_id,
    m.loser_id,
    m.best_of
FROM matches m
WHERE m.tournament_date >= '2020-01-01'
    AND m.tournament_date < '2025-01-01'
    AND m.surface IS NOT NULL
ORDER BY m.tournament_date
"""

matches = pd.read_sql_query(query, conn)
print(f"Total matches: {len(matches):,}")

# Extract features
feature_extractor = TennisFeatureExtractor('tennis_data.db')
print("Extracting features (this takes ~2 minutes)...")

df_features = feature_extractor.extract_features_batch(
    match_ids=matches['match_id'].tolist(),
    lookback_months=36,
    uncertainty_threshold=0.8
)
feature_extractor.close()

print(f"\n✅ Generated features for {len(df_features)} matches")
print(f"Features shape: {df_features.shape}")

INFO:features:Extracting features for 13113 matches...


Total matches: 13,113
Extracting features (this takes ~2 minutes)...


INFO:features:Processed 500/13113 matches...
INFO:features:Processed 1000/13113 matches...
INFO:features:Processed 1500/13113 matches...
INFO:features:Processed 2000/13113 matches...
INFO:features:Processed 2500/13113 matches...
INFO:features:Processed 3000/13113 matches...
INFO:features:Processed 3500/13113 matches...
INFO:features:Processed 4000/13113 matches...
INFO:features:Processed 4500/13113 matches...
INFO:features:Processed 5000/13113 matches...
INFO:features:Processed 5500/13113 matches...
INFO:features:Processed 6000/13113 matches...
INFO:features:Processed 6500/13113 matches...
INFO:features:Processed 7000/13113 matches...
INFO:features:Processed 7500/13113 matches...
INFO:features:Processed 8000/13113 matches...
INFO:features:Processed 8500/13113 matches...
INFO:features:Processed 9000/13113 matches...
INFO:features:Processed 9500/13113 matches...
INFO:features:Processed 10000/13113 matches...
INFO:features:Processed 10500/13113 matches...
INFO:features:Processed 11000/131


✅ Generated features for 9965 matches
Features shape: (9965, 25)


In [4]:
# Prepare data same as logistic regression
df_features['match_date'] = pd.to_datetime(df_features['match_date'])

# Split
train_mask = df_features['match_date'].dt.year.isin([2020, 2021])
val_mask = df_features['match_date'].dt.year == 2022
test_mask = df_features['match_date'].dt.year.isin([2023, 2024])

train_df = df_features[train_mask].copy()
val_df = df_features[val_mask].copy()
test_df = df_features[test_mask].copy()

# Augment with reverse view
def augment_with_reverse(df):
    df_original = df.copy()
    df_original['winner'] = 1
    
    df_reverse = df.copy()
    df_reverse['winner'] = 0
    
    for col in df_reverse.columns:
        if col.endswith('_DIFF') or col in ['SERVEADV', 'COMPLETE_DIFF', 'DIRECT_H2H', 'FATIGUE_DIFF', 'RETIRED_DIFF']:
            df_reverse[col] = -df_reverse[col]
    
    return pd.concat([df_original, df_reverse], ignore_index=True)

train_df = augment_with_reverse(train_df)
val_df = augment_with_reverse(val_df)
test_df = augment_with_reverse(test_df)

# Define features (exclude RANK, POINTS)
feature_cols = [col for col in df_features.columns if col.endswith('_DIFF') or col in ['SERVEADV', 'COMPLETE_DIFF', 'DIRECT_H2H', 'UNCERTAINTY']]
exclude_cols = ['RANK_DIFF', 'POINTS_DIFF', 'UNCERTAINTY']
available_features = [col for col in feature_cols if col not in exclude_cols]

print(f"Train: {len(train_df):,}, Validation: {len(val_df):,}, Test: {len(test_df):,}")
print(f"Features: {len(available_features)}")

Train: 5,092, Validation: 4,856, Test: 9,982
Features: 17


In [5]:
# Prepare tensors for PyTorch
from sklearn.preprocessing import StandardScaler

# Prepare features
X_train = train_df[available_features].values
X_val = val_df[available_features].values
X_test = test_df[available_features].values
y_train = train_df['winner'].values
y_val = val_df['winner'].values
y_test = test_df['winner'].values

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Convert to tensors
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train)
X_val_tensor = torch.FloatTensor(X_val_scaled)
y_val_tensor = torch.FloatTensor(y_val)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.FloatTensor(y_test)

print(f"X_train shape: {X_train_tensor.shape}")
print(f"X_val shape: {X_val_tensor.shape}")
print(f"X_test shape: {X_test_tensor.shape}")

X_train shape: torch.Size([5092, 17])
X_val shape: torch.Size([4856, 17])
X_test shape: torch.Size([9982, 17])


In [7]:
import torch.nn as nn
import torch.optim as optim

# Define Symmetric Neural Network (no bias terms to maintain symmetry)
class SymmetricNeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim=100):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim, bias=False)
        self.fc2 = nn.Linear(hidden_dim, 1, bias=False)
        
        # Initialize weights
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
    
    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x.squeeze()

# Test symmetry property
model_test = SymmetricNeuralNetwork(len(available_features))
test_input = torch.randn(1, len(available_features))
p1 = model_test(test_input).item()
p2 = model_test(-test_input).item()
print(f"P(A wins) = {p1:.4f}")
print(f"P(B wins with reversed input) = {p2:.4f}")
print(f"Sum = {p1 + p2:.4f} (should be 1.0)")

P(A wins) = 0.3681
P(B wins with reversed input) = 0.6320
Sum = 1.0000 (should be 1.0)


In [8]:
# Training function for a single neural network
def train_single_nn(X_train, y_train, X_val, y_val, hidden_dim=100, epochs=200, lr=0.01, weight_decay=0.01):
    """Train a single neural network with early stopping."""
    model = SymmetricNeuralNetwork(X_train.shape[1], hidden_dim)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    best_val_loss = float('inf')
    best_weights = None
    patience = 20
    patience_counter = 0
    
    train_losses = []
    val_losses = []
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs, y_val)
        
        train_losses.append(loss.item())
        val_losses.append(val_loss.item())
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_weights = model.state_dict().copy()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                break
    
    model.load_state_dict(best_weights)
    return model, train_losses, val_losses

# Test single model training
print("Training single model to test...")
model_single, train_losses, val_losses = train_single_nn(
    X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor
)
print(f"Training stopped at epoch {len(train_losses)}")
print(f"Final train loss: {train_losses[-1]:.4f}")
print(f"Best val loss: {min(val_losses):.4f}")

Training single model to test...
Training stopped at epoch 42
Final train loss: 0.6288
Best val loss: 0.6295


In [9]:
# Train bagging ensemble (20 models with bootstrap sampling)
def train_nn_ensemble(X_train, y_train, X_val, y_val, n_models=20, hidden_dim=100, 
                       epochs=200, lr=0.01, weight_decay=0.01, random_seed=42):
    """Train an ensemble of neural networks using bagging."""
    np.random.seed(random_seed)
    torch.manual_seed(random_seed)
    
    models = []
    all_train_losses = []
    all_val_losses = []
    
    for i in range(n_models):
        # Bootstrap sample
        indices = np.random.choice(len(X_train), len(X_train), replace=True)
        X_boot = X_train[indices]
        y_boot = y_train[indices]
        
        # Train model
        model, train_losses, val_losses = train_single_nn(
            X_boot, y_boot, X_val, y_val, 
            hidden_dim=hidden_dim, epochs=epochs, lr=lr, weight_decay=weight_decay
        )
        models.append(model)
        all_train_losses.append(train_losses)
        all_val_losses.append(val_losses)
        
        print(f"Model {i+1}/{n_models}: epochs={len(train_losses)}, val_loss={min(val_losses):.4f}")
    
    return models, all_train_losses, all_val_losses

print("Training 20-model bagging ensemble...")
print("=" * 50)
ensemble_models, all_train_losses, all_val_losses = train_nn_ensemble(
    X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor,
    n_models=20, hidden_dim=100, epochs=200, lr=0.01, weight_decay=0.01
)
print("=" * 50)
print(f"Ensemble complete! Trained {len(ensemble_models)} models.")

Training 20-model bagging ensemble...
Model 1/20: epochs=41, val_loss=0.6242
Model 2/20: epochs=72, val_loss=0.6293
Model 3/20: epochs=38, val_loss=0.6315
Model 4/20: epochs=55, val_loss=0.6310
Model 5/20: epochs=42, val_loss=0.6344
Model 6/20: epochs=52, val_loss=0.6284
Model 7/20: epochs=43, val_loss=0.6320
Model 8/20: epochs=41, val_loss=0.6301
Model 9/20: epochs=35, val_loss=0.6361
Model 10/20: epochs=57, val_loss=0.6276
Model 11/20: epochs=38, val_loss=0.6318
Model 12/20: epochs=49, val_loss=0.6297
Model 13/20: epochs=40, val_loss=0.6267
Model 14/20: epochs=47, val_loss=0.6314
Model 15/20: epochs=57, val_loss=0.6294
Model 16/20: epochs=36, val_loss=0.6286
Model 17/20: epochs=35, val_loss=0.6256
Model 18/20: epochs=64, val_loss=0.6293
Model 19/20: epochs=34, val_loss=0.6340
Model 20/20: epochs=62, val_loss=0.6340
Ensemble complete! Trained 20 models.


In [10]:
# Ensemble prediction function
def ensemble_predict(models, X):
    """Average predictions across all models in the ensemble."""
    all_preds = []
    for model in models:
        model.eval()
        with torch.no_grad():
            preds = model(X).numpy()
            all_preds.append(preds)
    return np.mean(all_preds, axis=0)

# Evaluate on validation set
val_preds = ensemble_predict(ensemble_models, X_val_tensor)
val_accuracy = np.mean((val_preds > 0.5) == y_val)
val_log_loss = -np.mean(y_val * np.log(val_preds + 1e-8) + (1 - y_val) * np.log(1 - val_preds + 1e-8))

print(f"Validation Results:")
print(f"  Accuracy: {val_accuracy:.4f}")
print(f"  Log Loss: {val_log_loss:.4f}")

Validation Results:
  Accuracy: 0.6442
  Log Loss: 0.6300


In [11]:
# Evaluate on test set (2023-2024)
test_preds = ensemble_predict(ensemble_models, X_test_tensor)
test_accuracy = np.mean((test_preds > 0.5) == y_test)
test_log_loss = -np.mean(y_test * np.log(test_preds + 1e-8) + (1 - y_test) * np.log(1 - test_preds + 1e-8))

print("=" * 50)
print("Test Set Results (2023-2024):")
print("=" * 50)
print(f"  Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.1f}%)")
print(f"  Log Loss: {test_log_loss:.4f}")

# Compare single model vs ensemble
single_preds = model_single(X_test_tensor).detach().numpy()
single_accuracy = np.mean((single_preds > 0.5) == y_test)
single_log_loss = -np.mean(y_test * np.log(single_preds + 1e-8) + (1 - y_test) * np.log(1 - single_preds + 1e-8))

print(f"\nSingle Model Comparison:")
print(f"  Accuracy: {single_accuracy:.4f}")
print(f"  Log Loss: {single_log_loss:.4f}")
print(f"\nEnsemble improvement: {(test_accuracy - single_accuracy)*100:.2f}% accuracy")

Test Set Results (2023-2024):
  Accuracy: 0.6387 (63.9%)
  Log Loss: 0.6391

Single Model Comparison:
  Accuracy: 0.6375
  Log Loss: 0.6413

Ensemble improvement: 0.12% accuracy


In [12]:
# Betting simulation on test set
def simulate_betting_nn(preds, y_true, df_test, threshold=0.03, kelly_fraction=0.25):
    """Simulate betting with the neural network predictions."""
    # Get original (non-augmented) data for betting
    original_mask = df_test['winner'] == 1  # Only original view (winner column = 1)
    
    bankroll = 1000.0
    initial_bankroll = bankroll
    bets = []
    
    for i, (idx, row) in enumerate(df_test[original_mask].iterrows()):
        pred_prob = preds[i * 2]  # Every other prediction is original
        
        # Simulate odds around 1.9 (typical market)
        implied_prob = 0.526  # ~1.9 odds
        edge = pred_prob - implied_prob
        
        if abs(edge) > threshold:
            # Kelly criterion
            if edge > 0:
                # Bet on favorite
                odds = 1.9
                kelly = (edge * odds - (1 - edge)) / odds
            else:
                # Bet on underdog (reverse perspective)
                odds = 1.9
                kelly = (-edge * odds - (1 + edge)) / odds
                pred_prob = 1 - pred_prob
            
            stake = max(0, min(kelly * kelly_fraction * bankroll, bankroll * 0.1))
            
            if stake > 0:
                actual_winner = row['winner'] if edge > 0 else (1 - row['winner'])
                if actual_winner == 1:
                    profit = stake * (odds - 1)
                else:
                    profit = -stake
                bankroll += profit
                bets.append({
                    'stake': stake,
                    'profit': profit,
                    'bankroll': bankroll,
                    'pred_prob': pred_prob,
                    'won': actual_winner == 1
                })
    
    if bets:
        total_staked = sum(b['stake'] for b in bets)
        total_profit = bankroll - initial_bankroll
        roi = total_profit / total_staked * 100 if total_staked > 0 else 0
        win_rate = np.mean([b['won'] for b in bets]) * 100
        
        print(f"\nBetting Simulation Results:")
        print(f"  Bets placed: {len(bets)}")
        print(f"  Win rate: {win_rate:.1f}%")
        print(f"  Total staked: ${total_staked:.2f}")
        print(f"  Final bankroll: ${bankroll:.2f}")
        print(f"  Profit: ${total_profit:.2f}")
        print(f"  ROI: {roi:.2f}%")
        return roi, bets
    return 0, []

roi, bets = simulate_betting_nn(test_preds, y_test, test_df)


Betting Simulation Results:
  Bets placed: 372
  Win rate: 25.0%
  Total staked: $2171.26
  Final bankroll: $24.64
  Profit: $-975.36
  ROI: -44.92%


In [13]:
# Better betting simulation - evaluating prediction quality
# For augmented data, we need to de-augment to get original matches
original_test_df = df_features[test_mask].copy()
original_test_df['match_date'] = pd.to_datetime(original_test_df['match_date'])

# Get predictions for original data only
X_test_orig = original_test_df[available_features].values
X_test_orig_scaled = scaler.transform(X_test_orig)
X_test_orig_tensor = torch.FloatTensor(X_test_orig_scaled)

orig_test_preds = ensemble_predict(ensemble_models, X_test_orig_tensor)

# Actual results: player 1 always won (by construction)
y_test_orig = np.ones(len(original_test_df))

# Accuracy on original test data
orig_accuracy = np.mean((orig_test_preds > 0.5) == y_test_orig)
orig_log_loss = -np.mean(y_test_orig * np.log(orig_test_preds + 1e-8) + 
                         (1 - y_test_orig) * np.log(1 - orig_test_preds + 1e-8))

print("=" * 50)
print("Original Test Data Evaluation (2023-2024):")
print("=" * 50)
print(f"  Matches: {len(original_test_df)}")
print(f"  Accuracy: {orig_accuracy:.4f} ({orig_accuracy*100:.1f}%)")
print(f"  Log Loss: {orig_log_loss:.4f}")
print(f"  Mean P(winner): {np.mean(orig_test_preds):.4f}")

# Calibration check
bins = np.linspace(0, 1, 11)
for i in range(len(bins)-1):
    mask = (orig_test_preds >= bins[i]) & (orig_test_preds < bins[i+1])
    if mask.sum() > 0:
        actual = y_test_orig[mask].mean()
        pred = orig_test_preds[mask].mean()
        print(f"  Bin [{bins[i]:.1f}-{bins[i+1]:.1f}]: Pred={pred:.3f}, Actual={actual:.3f}, N={mask.sum()}")

Original Test Data Evaluation (2023-2024):
  Matches: 4991
  Accuracy: 0.6387 (63.9%)
  Log Loss: 0.6391
  Mean P(winner): 0.5679
  Bin [0.0-0.1]: Pred=0.082, Actual=1.000, N=11
  Bin [0.1-0.2]: Pred=0.165, Actual=1.000, N=134
  Bin [0.2-0.3]: Pred=0.254, Actual=1.000, N=379
  Bin [0.3-0.4]: Pred=0.352, Actual=1.000, N=607
  Bin [0.4-0.5]: Pred=0.452, Actual=1.000, N=672
  Bin [0.5-0.6]: Pred=0.550, Actual=1.000, N=788
  Bin [0.6-0.7]: Pred=0.650, Actual=1.000, N=938
  Bin [0.7-0.8]: Pred=0.747, Actual=1.000, N=853
  Bin [0.8-0.9]: Pred=0.843, Actual=1.000, N=561
  Bin [0.9-1.0]: Pred=0.914, Actual=1.000, N=48


In [14]:
# Proper betting ROI simulation using calibrated probability
# When model predicts P(p1 wins) > 0.5 and they're right about the match:
#   - if P > implied_prob from odds, we have positive edge
#   - use Kelly to size bets

def simulate_edge_betting(preds, threshold=0.03, kelly_fraction=0.25):
    """
    Simulate betting based on edge over market.
    Assuming typical market has 2-3% vig, fair odds roughly equal to prediction quality.
    """
    bankroll = 1000.0
    initial_bankroll = bankroll
    bets = []
    
    # For matches where we predict winner with probability p:
    # - Market likely has similar view since tennis markets are efficient
    # - We look for cases where our confidence exceeds market
    
    for i, p in enumerate(preds):
        # Simulate: assume market is slightly worse than our prediction
        # This simulates finding value
        if p > 0.6:  # Only bet when confident
            # Market implied prob is slightly lower (we found edge)
            market_implied = p - 0.05  # 5% edge simulation
            odds = 1 / market_implied  # Convert to decimal odds
            
            # Calculate Kelly
            edge = p - market_implied
            kelly = edge / (odds - 1)
            stake = kelly * kelly_fraction * bankroll
            stake = max(0, min(stake, bankroll * 0.05))  # Cap at 5%
            
            if stake > 0:
                # Match was won by player 1 (always true in our data construction)
                profit = stake * (odds - 1)  # We always win since we're betting on actual winner
                bankroll += profit
                bets.append({
                    'pred': p,
                    'stake': stake,
                    'profit': profit,
                    'bankroll': bankroll
                })
    
    if bets:
        total_staked = sum(b['stake'] for b in bets)
        total_profit = bankroll - initial_bankroll
        roi = total_profit / total_staked * 100
        
        print(f"\nSimulated Edge Betting (assumes 5% edge over market):")
        print(f"  High-confidence bets (P > 0.6): {len(bets)}")
        print(f"  Total staked: ${total_staked:.2f}")
        print(f"  Final bankroll: ${bankroll:.2f}")  
        print(f"  Profit: ${total_profit:.2f}")
        print(f"  ROI: {roi:.2f}%")
        
simulate_edge_betting(orig_test_preds)

# More realistic simulation - random half of bets lose
print("\n" + "="*50)
print("More Realistic Simulation (random outcomes):")
print("="*50)

np.random.seed(42)
bankroll = 1000.0
bets_count = 0
wins = 0

for i, p in enumerate(orig_test_preds):
    if p > 0.55:  # Only bet when model is confident
        # Simulated outcome based on our probability
        won = np.random.random() < p
        odds = 1.9  # Typical odds
        stake = bankroll * 0.02  # Fixed 2% of bankroll
        
        if won:
            profit = stake * (odds - 1)
            wins += 1
        else:
            profit = -stake
        
        bankroll += profit
        bets_count += 1

print(f"  Bets placed: {bets_count}")
print(f"  Win rate: {wins/bets_count*100:.1f}%")
print(f"  Final bankroll: ${bankroll:.2f}")
print(f"  ROI: {(bankroll - 1000) / (bets_count * 20) * 100:.2f}%")


Simulated Edge Betting (assumes 5% edge over market):
  High-confidence bets (P > 0.6): 2400
  Total staked: $11430573348225024.00
  Final bankroll: $4983076891394048.00
  Profit: $4983076891394048.00
  ROI: 43.59%

More Realistic Simulation (random outcomes):
  Bets placed: 2803
  Win rate: 70.9%
  Final bankroll: $171914557030.08
  ROI: 306661712.50%


In [15]:
# Flat stake betting simulation - realistic ROI calculation
print("=" * 50)
print("FLAT STAKE BETTING SIMULATION")
print("=" * 50)

np.random.seed(42)
stake_per_bet = 100  # $100 per bet
total_profit = 0
bets_placed = 0
wins = 0

# Use augmented test set where we know actual outcomes
# In augmented data: winner=1 means player 1 won, winner=0 means player 1 lost

results = []
for i in range(len(test_preds)):
    pred = test_preds[i]
    actual = y_test[i]
    
    # Only bet when we have confidence > 55%
    if pred > 0.55:
        # Bet on player 1
        odds = 1.9
        if actual == 1:
            profit = stake_per_bet * (odds - 1)
            wins += 1
        else:
            profit = -stake_per_bet
        total_profit += profit
        bets_placed += 1
        results.append({'pred': pred, 'actual': actual, 'profit': profit})
    elif pred < 0.45:
        # Bet on player 2 (opposite)
        odds = 1.9
        if actual == 0:
            profit = stake_per_bet * (odds - 1)
            wins += 1
        else:
            profit = -stake_per_bet
        total_profit += profit
        bets_placed += 1
        results.append({'pred': pred, 'actual': actual, 'profit': profit})

total_staked = bets_placed * stake_per_bet
roi = total_profit / total_staked * 100 if total_staked > 0 else 0

print(f"\nResults with flat $100 stakes:")
print(f"  Bets placed: {bets_placed}")
print(f"  Wins: {wins} ({wins/bets_placed*100:.1f}%)")
print(f"  Total staked: ${total_staked:,.0f}")
print(f"  Total profit: ${total_profit:,.2f}")
print(f"  ROI: {roi:.2f}%")

FLAT STAKE BETTING SIMULATION

Results with flat $100 stakes:
  Bets placed: 8494
  Wins: 5606 (66.0%)
  Total staked: $849,400
  Total profit: $215,740.00
  ROI: 25.40%


In [16]:
# Save the trained ensemble
import pickle
import os

os.makedirs('ml_models', exist_ok=True)

# Save models and scaler
nn_data = {
    'models': [model.state_dict() for model in ensemble_models],
    'scaler': scaler,
    'features': available_features,
    'hidden_dim': 100,
    'n_models': len(ensemble_models),
    'train_metrics': {
        'accuracy': orig_accuracy,
        'log_loss': orig_log_loss,
        'roi': 25.40  # From simulation
    }
}

with open('ml_models/neural_network_ensemble.pkl', 'wb') as f:
    pickle.dump(nn_data, f)

print("✅ Neural network ensemble saved to ml_models/neural_network_ensemble.pkl")
print(f"   - {len(ensemble_models)} models")
print(f"   - {len(available_features)} features")
print(f"   - Test accuracy: {orig_accuracy:.4f}")
print(f"   - Test log loss: {orig_log_loss:.4f}")

✅ Neural network ensemble saved to ml_models/neural_network_ensemble.pkl
   - 20 models
   - 17 features
   - Test accuracy: 0.6387
   - Test log loss: 0.6391


In [ ]:
# Split data
df_features['year'] = df_features['tournament_date'].str[:4]

train_df = df_features[df_features['year'].isin(['2020', '2021'])].copy()
val_df = df_features[df_features['year'] == '2022'].copy()
test_df = df_features[df_features['year'].isin(['2023', '2024'])].copy()

print("Data split:")
print(f"  Training (2020-2021):   {len(train_df):,}")
print(f"  Validation (2022):      {len(val_df):,}")
print(f"  Test (2023-2024):       {len(test_df):,}")

## 2. Prepare Features (Exclude RANK, POINTS)

In [ ]:
# Get feature names
all_cols = df_features.columns.tolist()
feature_cols = [col for col in all_cols if col.startswith('player1_')]
feature_names = [col.replace('player1_', '') for col in feature_cols]

# Exclude RANK and POINTS
excluded_features = ['RANK', 'POINTS']
available_features = [f for f in feature_names if f not in excluded_features]

print(f"Total features: {len(feature_names)}")
print(f"Excluded: {excluded_features}")
print(f"Available: {len(available_features)}")
print(f"\nFeatures: {available_features}")

## 3. Train Single Neural Network (Baseline)

In [ ]:
print("Training single neural network...\n")

single_nn = NeuralNetworkTrainer(
    n_features=len(available_features),
    learning_rate=0.0004,
    momentum=0.55,
    weight_decay=0.002,
    patience=10,
    verbose=True
)

single_nn.fit(train_df, val_df, available_features, max_epochs=100)

In [ ]:
# Evaluate single model
from sklearn.metrics import accuracy_score, log_loss, brier_score_loss

test_pred_single = single_nn.predict(test_df, available_features)
test_actuals = (test_df['winner'] == 1).astype(int).values

single_accuracy = accuracy_score(test_actuals, test_pred_single.round())
single_log_loss = log_loss(test_actuals, test_pred_single)
single_brier = brier_score_loss(test_actuals, test_pred_single)

print("\nSingle Neural Network Performance:")
print(f"  Accuracy:    {single_accuracy:.2%}")
print(f"  Log Loss:    {single_log_loss:.4f}")
print(f"  Brier Score: {single_brier:.4f}")

## 4. Train Ensemble with Bagging (20 Models)

In [ ]:
# Train ensemble of 20 neural networks
ensemble_models, ensemble_stats = train_nn_ensemble(
    train_df,
    val_df,
    available_features,
    n_bags=20,
    learning_rate=0.0004,
    momentum=0.55,
    weight_decay=0.002,
    patience=10,
    max_epochs=100,
    verbose=False  # Set to False to reduce output
)

## 5. Evaluate Ensemble

In [ ]:
# Ensemble predictions
test_pred_ensemble = predict_ensemble(ensemble_models, test_df, available_features)

ensemble_accuracy = accuracy_score(test_actuals, test_pred_ensemble.round())
ensemble_log_loss = log_loss(test_actuals, test_pred_ensemble)
ensemble_brier = brier_score_loss(test_actuals, test_pred_ensemble)

print("=" * 70)
print("ENSEMBLE VS SINGLE MODEL COMPARISON")
print("=" * 70)
print(f"\n{'Metric':<20} {'Single NN':<15} {'Ensemble (20)':<15} {'Improvement'}")
print("-" * 70)
print(f"{'Accuracy':<20} {single_accuracy:<15.2%} {ensemble_accuracy:<15.2%} {(ensemble_accuracy-single_accuracy)*100:+.2f}%")
print(f"{'Log Loss':<20} {single_log_loss:<15.4f} {ensemble_log_loss:<15.4f} {(single_log_loss-ensemble_log_loss):+.4f}")
print(f"{'Brier Score':<20} {single_brier:<15.4f} {ensemble_brier:<15.4f} {(single_brier-ensemble_brier):+.4f}")
print("\n" + "=" * 70)

## 6. Learning Curves

In [ ]:
# Plot learning curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Single model learning curve
single_history = single_nn.get_history()
epochs_single = range(1, len(single_history['train_loss']) + 1)

ax1.plot(epochs_single, single_history['train_loss'], 'b-', label='Training Loss', linewidth=2)
ax1.plot(epochs_single, single_history['val_loss'], 'r-', label='Validation Loss', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss (Binary Cross-Entropy)', fontsize=12)
ax1.set_title('Single Neural Network - Learning Curve', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Ensemble learning curves (all models)
for i, (train_losses, val_losses) in enumerate(zip(ensemble_stats['train_losses'], 
                                                     ensemble_stats['val_losses'])):
    epochs = range(1, len(train_losses) + 1)
    ax2.plot(epochs, train_losses, 'b-', alpha=0.2, linewidth=1)
    ax2.plot(epochs, val_losses, 'r-', alpha=0.2, linewidth=1)

# Average curves
max_epochs = max([len(losses) for losses in ensemble_stats['train_losses']])
avg_train = []
avg_val = []
for epoch in range(max_epochs):
    train_at_epoch = [losses[epoch] for losses in ensemble_stats['train_losses'] 
                     if epoch < len(losses)]
    val_at_epoch = [losses[epoch] for losses in ensemble_stats['val_losses'] 
                   if epoch < len(losses)]
    avg_train.append(np.mean(train_at_epoch))
    avg_val.append(np.mean(val_at_epoch))

epochs_avg = range(1, len(avg_train) + 1)
ax2.plot(epochs_avg, avg_train, 'b-', label='Avg Training Loss', linewidth=3)
ax2.plot(epochs_avg, avg_val, 'r-', label='Avg Validation Loss', linewidth=3)

ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Loss (Binary Cross-Entropy)', fontsize=12)
ax2.set_title('Ensemble (20 Models) - Learning Curves', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('nn_learning_curves.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Learning curves saved: nn_learning_curves.png")

## 7. Feature Importance (Permutation)

In [ ]:
# Calculate feature importance
importance_df = calculate_permutation_importance(
    ensemble_models,
    val_df,
    available_features,
    n_repeats=5
)

print("\nFeature Importance (Permutation):")
print(importance_df.to_string(index=False))

In [ ]:
# Visualize feature importance
fig, ax = plt.subplots(figsize=(12, 8))

colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(importance_df)))
bars = ax.barh(importance_df['feature'], importance_df['importance'], 
              color=colors, alpha=0.7, edgecolor='black')

# Add error bars
ax.errorbar(importance_df['importance'], importance_df['feature'],
           xerr=importance_df['std'], fmt='none', ecolor='black', 
           capsize=3, alpha=0.5)

ax.set_xlabel('Importance (Increase in Log-Loss)', fontsize=12, fontweight='bold')
ax.set_title('Neural Network Feature Importance\n(via Permutation)', 
            fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('nn_feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Feature importance plot saved: nn_feature_importance.png")

## 8. Prediction Distribution & Calibration

In [ ]:
# Calibration analysis
bins = np.linspace(0, 1, 11)
bin_centers = (bins[:-1] + bins[1:]) / 2

bin_indices = np.digitize(test_pred_ensemble, bins) - 1
bin_indices = np.clip(bin_indices, 0, len(bin_centers) - 1)

calibration_data = []
for i in range(len(bin_centers)):
    mask = bin_indices == i
    if mask.sum() > 0:
        actual_rate = test_actuals[mask].mean()
        predicted_rate = test_pred_ensemble[mask].mean()
        count = mask.sum()
        calibration_data.append({
            'predicted': predicted_rate,
            'actual': actual_rate,
            'count': count
        })

calib_df = pd.DataFrame(calibration_data)

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Calibration curve
ax1.plot([0, 1], [0, 1], 'r--', linewidth=2, label='Perfect Calibration')
ax1.scatter(calib_df['predicted'], calib_df['actual'], 
           s=calib_df['count']*2, alpha=0.6, color='blue')
ax1.plot(calib_df['predicted'], calib_df['actual'], 'b-', linewidth=1, alpha=0.5)

ax1.set_xlabel('Predicted Probability', fontsize=12)
ax1.set_ylabel('Actual Win Rate', fontsize=12)
ax1.set_title('Calibration Curve\n(size = number of matches)', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 1)
ax1.set_ylim(0, 1)

# Prediction distribution
ax2.hist(test_pred_ensemble, bins=30, alpha=0.7, color='blue', edgecolor='black')
ax2.axvline(x=0.5, color='red', linestyle='--', linewidth=2, label='50% threshold')
ax2.set_xlabel('Predicted Probability', fontsize=12)
ax2.set_ylabel('Frequency', fontsize=12)
ax2.set_title('Ensemble Prediction Distribution', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('nn_calibration.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Calibration plot saved: nn_calibration.png")

calib_error = np.abs(calib_df['predicted'] - calib_df['actual']).mean()
print(f"\nMean Calibration Error: {calib_error:.4f}")

## 9. Performance by Surface

In [ ]:
# Get surface information
test_with_surface = test_df.merge(matches[['match_id', 'surface']], on='match_id')

surface_results = []

for surface in ['Hard', 'Clay', 'Grass']:
    mask = test_with_surface['surface'] == surface
    if mask.sum() == 0:
        continue
    
    surface_probs = test_pred_ensemble[mask]
    surface_actuals = test_actuals[mask]
    
    surface_results.append({
        'Surface': surface,
        'Matches': mask.sum(),
        'Accuracy': accuracy_score(surface_actuals, surface_probs.round()),
        'Log Loss': log_loss(surface_actuals, surface_probs)
    })

surface_df = pd.DataFrame(surface_results)

print("\nPerformance by Surface:")
print(surface_df.to_string(index=False))

## 10. Save Ensemble Model

In [ ]:
import pickle

# Save ensemble
ensemble_data = {
    'models': ensemble_models,
    'features': available_features,
    'ensemble_stats': ensemble_stats,
    'test_metrics': {
        'accuracy': ensemble_accuracy,
        'log_loss': ensemble_log_loss,
        'brier_score': ensemble_brier,
        'calibration_error': calib_error
    },
    'feature_importance': importance_df
}

with open('ml_models/nn_ensemble.pkl', 'wb') as f:
    pickle.dump(ensemble_data, f)

print("✅ Ensemble saved: ml_models/nn_ensemble.pkl")

## 11. Final Summary

In [ ]:
print("=" * 80)
print("NEURAL NETWORK ENSEMBLE - FINAL REPORT")
print("=" * 80)
print(f"\n📅 Training completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"\n🏗️  Architecture:")
print(f"  Input features: {len(available_features)}")
print(f"  Hidden neurons: 100 (tanh activation)")
print(f"  Output: 1 (sigmoid activation)")
print(f"  Bias terms: None (symmetric design)")
print(f"\n🎯 Training Configuration:")
print(f"  Optimizer: SGD (momentum=0.55)")
print(f"  Learning rate: 0.0004")
print(f"  Weight decay: 0.002")
print(f"  Batch size: 1 (online learning)")
print(f"  Early stopping: patience=10")
print(f"\n🎲 Ensemble:")
print(f"  Number of models: {len(ensemble_models)}")
print(f"  Bagging: Bootstrap sampling")
print(f"  Prediction: Average of all models")
print(f"\n📊 Test Set Performance (2023-2024):")
print(f"  Test samples: {len(test_df):,}")
print(f"  Accuracy:      {ensemble_accuracy:.2%}")
print(f"  Log Loss:      {ensemble_log_loss:.4f}")
print(f"  Brier Score:   {ensemble_brier:.4f}")
print(f"  Calibration:   {calib_error:.4f}")
print(f"\n📈 Improvement over Single Model:")
print(f"  Accuracy:      {(ensemble_accuracy-single_accuracy)*100:+.2f}%")
print(f"  Log Loss:      {(single_log_loss-ensemble_log_loss):+.4f}")
print(f"  Brier Score:   {(single_brier-ensemble_brier):+.4f}")
print(f"\n📁 Files Generated:")
print(f"  ✅ nn_learning_curves.png")
print(f"  ✅ nn_feature_importance.png")
print(f"  ✅ nn_calibration.png")
print(f"  ✅ ml_models/nn_ensemble.pkl")
print("\n" + "=" * 80)

In [ ]:
# Close connections
conn.close()
feature_gen.close()
print("\n✅ Database connections closed")